In [2]:
import numpy as np
import random

In [22]:
#Sparse signal 
n = 100
p = [10, 25, 50]
rho = [0, 0.25, 0.5]
beta_1 = np.array([(2/np.sqrt(n)) if j <= np.sqrt(p[0]) else 0 for j in range(1, p[0] + 1)]).re

In [5]:
j = 3
j <= 3

True

In [23]:
print(beta_1)

[0.2 0.2 0.2 0.  0.  0.  0.  0.  0.  0. ]
